In [1]:
import numpy as np
import pandas as pd
import re
import pickle

In [2]:
def read_pickle(file):
    data = []
    try:
        while True:
            data.append(pickle.load(file))
    except EOFError:
        pass
    return data


with open('flats_data.pickle', 'rb') as f:
    DF = pd.DataFrame(read_pickle(f))

In [3]:
DF.count()

Cсылка                 1113378
Цена                   1113378
Время загрузки         1113378
Ближайшее метро        1113378
Адрес                  1112017
Этаж                   1113148
Площадь кухни           939012
Расстояние до метро    1113378
Количество комнат      1108678
Описание               1113346
Жилая площадь           893688
Вид объекта            1113378
Площадь                1113149
Город                  1113378
Регион                 1113378
ID                     1113378
Тип объявления         1113378
Этажей в доме          1113129
Срок сдачи              206557
lat                    1113376
lng                    1113376
Тип дома                 85483
Название ЖК             192000
О доме                  148742
О квартире              148742
dtype: int64

In [4]:
def get_remont(string):
    if re.search('ремонт|отделк', string.lower()):
        return string.lower()

def get_year_of_const(string):
    if re.search('постройки|г[.]п[.]', string.lower()):
        return string.lower()

def get_view(string):
    if re.search('вид |окон|окна', string.lower()):
        return string.lower()

def get_balkony(string):
    if re.search('лоджи|балкон', string.lower()):
        return string.lower()

def get_ceiling(string):
    if re.search('потол', string.lower()):
        return string.lower()

def get_type_bath(string):
    if re.search(r'сануз|с/у', string.lower()):
        return string.lower()

def get_consierj(string):
    if len(re.findall('консьерж', string.lower())) > 0:
        return 1

def get_house_type(string):
    search = re.findall('панельный|монолитный|кирпичный|блочный|деревянный', string.lower())
    if len(search) > 0:
        return search[0]

def get_layout(string):
    if re.search('планировк', string.lower()):
        return string.lower()
    
def get_add_info(description):
    string_list = re.split('[.,!;][\s]|[\n]', description)
    cols = ['Ремонт', 'Год постройки', 'Вид из окон', 'Балкон или лоджия', 
            'Высота потолков', 'Санузел', 'Консьерж', 'Тип дома', 'Планировка']
    functions = dict(zip(cols, [get_remont, get_year_of_const, get_view, get_balkony, 
                                get_ceiling, get_type_bath, get_consierj, get_house_type, get_layout]))
    dict_ind = dict([(col, True) for col in cols])
    dict_params = {}
    for string in string_list:
        for col, ind in dict_ind.items():
            if ind:
                result = functions[col](string)
                if result is not None:
                    dict_params[col] = result
                    dict_ind[col] = False
    return dict_params

def get_add_info_df(Series):
    return pd.DataFrame([get_add_info(desc) for desc in Series.astype(str)])

In [5]:
descr_DF = get_add_info_df(DF['Описание'])

In [6]:
descr_DF.count()

Вид из окон          494213
Высота потолков      222763
Ремонт               669070
Планировка           307030
Балкон или лоджия    378454
Санузел              393150
Консьерж             108924
Год постройки         80594
Тип дома              86335
dtype: int64

In [7]:
def get_findall(pattern, s, ans):
    if type(s) == str:
        find_list = re.findall(pattern, s)
        if len(find_list) > 0:
            return [ans]
    return s
    
def clean_val(obj):
    if type(obj) == list and len(obj) > 0:
        return obj[0]
    else:
        return None
    
def proc_desc_df(df):
    df = df.copy()
    df['Ремонт'] = df['Ремонт'].map(lambda s: get_findall('с отделк|с ремонт|сделан|чистов', s, 'есть'), na_action='ignore')
    df['Ремонт'] = df['Ремонт'].map(lambda s: get_findall('чернов| нет|требует|без |отсутств', s, 'требуется'), na_action='ignore')
    df['Ремонт'] = df['Ремонт'].map(lambda s: get_findall('косметическ', s, 'косметический'), na_action='ignore')
    df['Ремонт'] = df['Ремонт'].map(lambda s: get_findall('отличн|современ|хорош|качеств|евро|свежи', s, 'евро'), na_action='ignore')
    df['Ремонт'] = df['Ремонт'].map(lambda s: get_findall('авторск|дизайн', s, 'дизайнерский'), na_action='ignore')
    df['Ремонт'] = df['Ремонт'].map(lambda s: get_findall('капитальн', s, 'капитальный'), na_action='ignore')
    
    df['Вид из окон'] = df['Вид из окон'].map(lambda s: get_findall('сквер|парк|реку', s, 'на природу'), na_action='ignore')
    df['Вид из окон'] = df['Вид из окон'].map(lambda s: get_findall('двор', s, 'во двор'), na_action='ignore')
    df['Вид из окон'] = df['Вид из окон'].map(lambda s: get_findall('на улиц', s, 'на улицу'), na_action='ignore')
    df['Вид из окон'] = df['Вид из окон'].map(lambda s: get_findall('живопис|красив|прекрасн|великолеп|хорош', s, 'красивый'), na_action='ignore')
    
    df['Балкон или лоджия'] = df['Балкон или лоджия'].map(lambda s: get_findall(' нет|отсутств', s, 'нет'), na_action='ignore')
    df['Балкон или лоджия'] = df['Балкон или лоджия'].map(lambda s: get_findall('балкон', s, 'балкон'), na_action='ignore')
    df['Балкон или лоджия'] = df['Балкон или лоджия'].map(lambda s: get_findall('лоджи', s, 'лоджия'), na_action='ignore')
    
    df['Высота потолков'] = df['Высота потолков'].map(lambda s: re.findall('\d{1,2}[.,]?\d{,2}', s), na_action='ignore')   
    df['Год постройки'] = df['Год постройки'].map(lambda s: re.findall('\d{2,4}', s), na_action='ignore')
    
    df['Санузел'] = df['Санузел'].map(lambda s: get_findall('совме|смежн', s, 'совмещённый'), na_action='ignore')
    df['Санузел'] = df['Санузел'].map(lambda s: get_findall('изолир|раздел', s, 'раздельный'), na_action='ignore')
    
    df['Планировка'] = df['Планировка'].map(lambda s: get_findall('совмещ|смежн', s, 'смежные'), na_action='ignore')
    df['Планировка'] = df['Планировка'].map(lambda s: get_findall('изолир|раздел', s, 'изолированные'), na_action='ignore')
    df['Планировка'] = df['Планировка'].map(lambda s: get_findall('распашон', s, 'распашонка'), na_action='ignore')
    df['Планировка'] = df['Планировка'].map(lambda s: get_findall('линей', s, 'линейка'), na_action='ignore')
    df['Планировка'] = df['Планировка'].map(lambda s: get_findall('евро', s, 'евродвушка'), na_action='ignore')
    df['Планировка'] = df['Планировка'].map(lambda s: get_findall('студ', s, 'студия'), na_action='ignore')
    
    for col in ['Ремонт', 'Год постройки', 'Вид из окон', 'Балкон или лоджия', 'Высота потолков', 'Санузел', 'Планировка']:
        df[col] = df[col].map(clean_val, na_action='ignore') 
    
    return df

In [8]:
new_descr_DF = proc_desc_df(descr_DF)

In [9]:
descr_DF.count() - new_descr_DF.count()

Вид из окон          295394
Высота потолков       76070
Ремонт               132162
Планировка           257691
Балкон или лоджия         0
Санузел              183121
Консьерж                  0
Год постройки         14086
Тип дома                  0
dtype: int64

In [11]:
import json

def str_to_json(string):
    if type(string) == str:
        try:
            return json.loads(string.replace("'", '"').replace('\\xa0', ' '))
        except:
            return {}
    elif type(string) == dict:
        return dict([(key,value.replace('\\xa0', ' ')) for key, value in string.items()])
    else:
        return {}
    
def get_view_value(string):
    find = [pattern.lower() for pattern in ['На солнечную сторону', 'На улицу', 'Во двор'] if re.search(pattern, string)]
    if len(find) > 0:
        return find[0]

def get_year_from_col(s):
    if type(s) == str:
        year = re.findall('d{4}', s) 
        if len(year) > 0:
            return year[0]
        else:
            return np.nan
    else:
        return s

def get_year(y):
    if y < 23:
        return 2000+y
    elif y < 100:
        return 1900+y
    elif  1800 < y < 2023:
        return y
    else:
        return np.nan

def prepr_metro(string):
    string = re.sub('[(].+[)]|станция ','', string).strip().lower()
    for item, value in zip(['пр-т', 'б-р', 'им.', 'ё', 'ул. ', 'ул.', 'ш.', 'пл. ', 'пл.' 'ниетест'], 
                           ['проспект', 'бульвар', 'имени', 'е', 'улица ', 'улица ', 'шоссе ', 'площадь', 'площадь ', 'ние']):
        string = string.replace(item, value)
    return string

def transform_df(df, descr_df):
    df = df.copy()
    add_df = pd.concat([pd.DataFrame(list(df[col].map(lambda x: str_to_json(x)).values)) 
                        for col in ['О доме', 'О квартире']], axis = 1).rename(columns = 
                {'Общая площадь': 'Площадь', 'Высота потолков, м': 'Высота потолков', 'Окна': 'Вид из окон'})
    #print(add_df)
    for col in ['Двор', 'Мебель', 'Техника', 'В доме', 'Парковка']:
        for val in set(', '.join([i for i in add_df[col].unique() if type(i) == str]).split(', ')):
            add_df[val] = add_df[col].map(lambda s: 1 if len(re.findall(val, s)) > 0 else 0, na_action='ignore')
        add_df = add_df.drop(columns=col)
    add_df['Вид из окон'] = add_df['Вид из окон'].map(get_view_value, na_action='ignore')
    add_df['Планировка'] = descr_df['Планировка'].fillna(add_df['Тип комнат'])
    add_df = add_df.drop(columns = 'Тип комнат')
    
    for col in descr_df.columns:
        if col in add_df.columns:
            add_df[col] = add_df[col].map(lambda s: s.lower() if type(s)==str else s, 
                                          na_action = 'ignore').fillna(descr_df[col])
        else:
            add_df[col] = descr_df[col]
            
    for col in add_df.columns:
        if col in df.columns:
            df[col] = df[col].map(lambda s: s.lower() if type(s)==str else s, 
                                          na_action = 'ignore').fillna(add_df[col])
        else:
            df[col] = add_df[col]
    
    df['Тёплый пол'] = df['Тёплый пол'].replace('Есть', 1).fillna(0)
    df['Высота потолков'] = df['Высота потолков'].map(lambda s: s.replace(',', '.') 
                        if type(s)==str else s).astype(float).map(lambda h: h if h < 10 else np.nan)
    
    df['Год постройки'] = df['Год постройки'].astype(float, errors = 'ignore').map(lambda s:
        get_year(s), na_action='ignore').fillna(df['Срок сдачи'].map(lambda s: get_year_from_col(s), 
                                        na_action = 'ignore')).astype(float, errors = 'ignore')
    df['Запланирован снос'] = df['Запланирован снос'].replace('Да', 1).fillna(0)
    
    df['Ближайшее метро'] = df['Ближайшее метро'].map(prepr_metro, na_action = 'ignore')
    df = df.drop(columns = ['О доме', 'О квартире', 'Название ЖК', 'Плита', 'Микроволновка', 'Срок сдачи', 'Описание'])
    
    return df

In [12]:
new_DF = transform_df(DF, new_descr_DF)

In [ ]:
new_DF

In [13]:
new_DF.count()

Cсылка                     1113378
Цена                       1113378
Время загрузки             1113378
Ближайшее метро            1113378
Адрес                      1112017
Этаж                       1113148
Площадь кухни               939012
Расстояние до метро        1113378
Количество комнат          1108678
Жилая площадь               893688
Вид объекта                1113378
Площадь                    1113149
Город                      1113378
Регион                     1113378
ID                         1113378
Тип объявления             1113378
Этажей в доме              1113129
lat                        1113376
lng                        1113376
Тип дома                    296979
Год постройки               217239
Пассажирский лифт            85841
Грузовой лифт                60788
Запланирован снос          1113378
Санузел                     320445
Ремонт                      610802
Статус                      148742
Высота потолков             195845
Вид из окон         

In [14]:
new_DF.to_csv('preprocessing_flats.csv')